In [2]:
import openai
import os
import json

# Get the API key from environment variables
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

# Clean the Corpus copy.txt

In [49]:
import re
corpus = open('corpus copy human.txt', 'r').read()
# split the corpus at blank lines
corpus = re.split('\n\s*\n', corpus)
corpus = [s for s in corpus if len(s) > 0] # remove empty sentences
display(corpus[:5])
# save the corpus to a csv
with open('corpus_list.txt', 'w') as f:
    f.write('\n'.join(corpus))

['Virginia Tech (VT), formally the Virginia Polytechnic Institute and State University (VPI),[9] is a public land-grant research university with its main campus in Blacksburg, Virginia. It also has educational facilities in six regions statewide, a research center in Punta Cana, Dominican Republic, and a study-abroad site in Riva San Vitale, Switzerland. Through its Corps of Cadets ROTC program, Virginia Tech is a senior military college.[10]\nVirginia Tech offers 280 undergraduate and graduate degree programs to its 37,000 students; as of 2016, it was the state\'s second-largest public university by enrollment.[11] It is classified among "R1: Doctoral Universities – Very high research activity".[12]\nThe university\'s athletic teams are known as the Virginia Tech Hokies and compete in Division I of the NCAA as members of the Atlantic Coast Conference. VT alumni include 2 Rhodes Scholars, 4 Marshall Scholars, 38 Goldwater Scholars, and 131 Fulbright Scholars.[a][14][15][16] Among its a

# Extract QAs using ChatGPT

In [35]:
core_messages = []
core_messages.append({'role': 'system', 'content': """
                 You will be given a paragraph related to the college Virginia Tech. Your task is to generate question and answers related to Virginia Tech based on the sentence. You must return the questions and answers in a list of python dictionaries, or equivalently, a list of JSONs. An example of this format is given between the triple backticks ```[{"question": "What is the capital of France?", "answer": "Paris"}, {"question": "When was Harvard founded?", "answer": "1636"}]```
                 """})

In [54]:
qas = []
start_index = 21
for i, p in enumerate(corpus[start_index:]):
    print(f'Paragraph: {i+start_index} of {len(corpus)}')
    temp_messages = core_messages + [{'role': 'user', 'content': p}]
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=temp_messages,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        temperature=0.7,
        stream=False,
    )
    try:
        d = json.loads(response.choices[0].message['content'])
        qas += d
        with open('qas.json', 'w') as file:
            json.dump(qas, file, indent=4)
    except: 
        print(f'Error in parsing response. Skipping paragraph {i+start_index}')
        pass

Paragraph: 21 of 80
Paragraph: 22 of 80
Paragraph: 23 of 80
Paragraph: 24 of 80
Paragraph: 25 of 80
Paragraph: 26 of 80
Paragraph: 27 of 80
Paragraph: 28 of 80
Paragraph: 29 of 80
Paragraph: 30 of 80
Paragraph: 31 of 80
Paragraph: 32 of 80
Paragraph: 33 of 80
Paragraph: 34 of 80
Paragraph: 35 of 80
Paragraph: 36 of 80
Paragraph: 37 of 80
Paragraph: 38 of 80
Paragraph: 39 of 80
Paragraph: 40 of 80
Paragraph: 41 of 80
Paragraph: 42 of 80
Paragraph: 43 of 80
Paragraph: 44 of 80
Paragraph: 45 of 80
Paragraph: 46 of 80
Paragraph: 47 of 80
Paragraph: 48 of 80
Paragraph: 49 of 80
Paragraph: 50 of 80
Paragraph: 51 of 80
Paragraph: 52 of 80
Paragraph: 53 of 80
Paragraph: 54 of 80
Error in parsing response. Skipping paragraph 54
Paragraph: 55 of 80
Paragraph: 56 of 80
Paragraph: 57 of 80
Paragraph: 58 of 80
Paragraph: 59 of 80
Paragraph: 60 of 80
Paragraph: 61 of 80
Paragraph: 62 of 80
Paragraph: 63 of 80
Paragraph: 64 of 80
Paragraph: 65 of 80
Paragraph: 66 of 80
Paragraph: 67 of 80
Paragraph: 

In [55]:
# Compile all of the json files that start with "qas" into one big json file
import glob
import json
qas = []
for file in glob.glob("qas*.json"):
    with open(file, 'r') as f:
        qas += json.load(f)

with open('qas_final_v2.json', 'w') as file:
    json.dump(qas, file, indent=4)

In [56]:
len(qas)

1448

In [4]:
# Turn the final json file into a csv
import pandas as pd
df = pd.read_json('qas_final_v2.json')
df.to_csv('qas_final_v2.csv')

# Export the Annotated Data

In [8]:
import pandas as pd
df = pd.read_csv('annotated_data_1_.csv')
print(len(df))
df.head()

1445


,id,question,answer,label
0,0,What is the full name of Virginia Tech?,Virginia Polytechnic Institute and State Unive...,1.0
1,1,Where is the main campus of Virginia Tech loca...,"Blacksburg, Virginia",1.0
2,2,How many students does Virginia Tech have?,"37,000",1.0
3,3,What is the classification of Virginia Tech am...,R1: Doctoral Universities - Very high research...,1.0
4,4,What are the athletic teams of Virginia Tech c...,Virginia Tech Hokies,1.0


In [9]:
# check for rows with empty values
df.isnull().sum()

id          0
question    0
answer      0
label       0
dtype: int64

In [10]:
display(df[df['label'] == 0])

,id,question,answer,label
199,199,When will Matthew M. Winston Jr. start his new...,On November 1.,0.0
250,250,What did President Minor say was impossible to...,all that is to be desired,0.0
314,314,What is the name of the college mentioned in t...,Virginia Tech,0.0
315,315,What is the topic of the paragraph?,Senior Year,0.0
316,316,What subject is mentioned in the paragraph?,Book-Keeping,0.0
...,...,...,...,...
1205,1205,Who else registered on the same day?,Mic,0.0
1206,1206,How many students were matriculated during the...,30,0.0
1225,1225,Until when did students have the option to eat...,1881,0.0
1299,1299,Who was Virginia Tech's director of athletics ...,Frank Moseley,0.0


In [6]:
# filter out the rows with 0 labels, count, and export to csv
df_filtered = df[df['label'] == 1]
print(len(df_filtered))
df_filtered.to_csv('data_filtered.csv')

1195


# Data Augmentation

In [6]:
import pandas as pd
import nlpaug.augmenter.word as naw

In [4]:
# import the data
data = pd.read_csv('data/data_filtered.csv').drop(columns=['Unnamed: 0', 'id', 'label'])
data.head()

,question,answer
0,What is the full name of Virginia Tech?,Virginia Polytechnic Institute and State Unive...
1,Where is the main campus of Virginia Tech loca...,"Blacksburg, Virginia"
2,How many students does Virginia Tech have?,"37,000"
3,What is the classification of Virginia Tech am...,R1: Doctoral Universities - Very high research...
4,What are the athletic teams of Virginia Tech c...,Virginia Tech Hokies


In [8]:
back_translation_aug = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-de',
    to_model_name='facebook/wmt19-de-en'
)

Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-en-de and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-de-en and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
data['question'] = back_translation_aug.augment(data['question'])
data['answer'] = back_translation_aug.augment(data['answer'])

ValueError: Length of values (1) does not match length of index (1195)

In [6]:
# read lines of data from a text file
import json
with open('7b_training_data.txt', 'r') as f:
    corpus = f.readlines()

corpus = [json.loads(s.strip().replace("'", '"')) for s in corpus]

# dump the dictionary to a json file
with open('7b_training_data.json', 'w') as f:
    json.dump(corpus, f, indent=4)